## DACON  풍력발전량 예측

독립변수 중 범주형 변수는 원핫으로 바꾸고, 정규화 하여 automl 사용
* 추가
* 1.독립변수에 스케일링 적용
* 2.독립변수에 스케일링 및 종속변수에 로그변환 적용
* 3.종속변수에 로그변환 적용
* 4.독립변수 그대로 사용 
* 위 방식들 중 1의 결과가 가장 좋아서 1의 방식으로 진행



In [ ]:
import sys
print(sys.version)

3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]


In [ ]:
! pip install -U pip
! pip install -U setuptools wheel

#Install the proper version of PyTorch following https://pytorch.org/get-started/locally/
! pip install torch==1.12.0+cu113 torchvision==0.13.0+cu113 torchtext==0.13.0 --extra-index-url https://download.pytorch.org/whl/cu113

! pip install autogluon

In [ ]:
!pip install -U pip
!pip install -U setuptools wheel
!pip install torch==1.13.1+cpu torchvision==0.14.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html

!pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached setuptools-67.4.0-py3-none-any.whl (1.1 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.4.0 which is incompatible.
Looking in indexes: h

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#base
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

#scaling
from sklearn.preprocessing import StandardScaler

#module
from sklearn.model_selection import  KFold, train_test_split, cross_val_score, cross_validate

#scoring
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

#data_path
test_path = '/content/test.csv'
train_path = '/content/train.csv'

#one-hot
from sklearn.preprocessing import OneHotEncoder

#automl
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
data = pd.read_csv(train_path).drop(columns = ["id"]) 
test = pd.read_csv(test_path).drop(columns = ["id"])

In [ ]:
encoder=OneHotEncoder()

In [ ]:
onehot = encoder.fit_transform(data[['snowing']])
onehot_te = encoder.fit_transform(test[['snowing']])

In [ ]:
onehot = onehot.toarray()
onehot_te = onehot_te.toarray()

In [ ]:
onehot = pd.DataFrame(onehot)
onehot_te = pd.DataFrame(onehot_te)
onehot.columns = ['False','True']
onehot_te.columns = ['False','True']

In [ ]:
data = data.drop(columns='snowing')
test = test.drop(columns='snowing')

In [ ]:
data = pd.concat([data,onehot],axis=1)
test = pd.concat([test,onehot_te],axis=1)

In [ ]:
data_y = data["target"]
data_x = data.drop(columns='target')

col_names = list(data_x.columns)

s_scaler = StandardScaler()
data_x_scaled = s_scaler.fit_transform(data_x)
test_x_scaled = s_scaler.fit_transform(test)

In [ ]:
data_x = pd.DataFrame(data_x_scaled, columns=col_names) 
test = pd.DataFrame(test_x_scaled, columns=col_names) 

In [ ]:
data = pd.concat([data_x,data_y],axis=1)

In [ ]:
test.head()

,temperature,pressure,humidity,wind_speed,wind_direction,precipitation,cloudiness,False,True
0,0.482146,-0.019144,0.505322,-0.362943,-1.085194,-0.388975,-0.145871,0.083355,-0.083355
1,0.200084,-0.021691,-0.630261,-0.070069,0.219150,0.215914,0.388729,0.083355,-0.083355
2,1.762338,-0.019298,-1.306204,0.076367,-1.555039,2.635473,-0.145871,0.083355,-0.083355
3,1.010854,-0.021614,-0.535629,-0.509380,-0.853778,-0.388975,-0.600850,0.083355,-0.083355
4,-0.645915,-0.022849,0.126795,-0.509380,-0.994030,0.215914,0.479725,0.083355,-0.083355


In [ ]:
train_data = TabularDataset(data)
test_data = TabularDataset(test)

In [ ]:
predictor = TabularPredictor(label= 'target', eval_metric='mean_absolute_error').fit(train_data, presets= 'high_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20230226_142903/"
Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230226_142903/"
AutoGluon Version:  0.7.0
Python Version:     3.8.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Dec 10 16:00:40 UTC 2022
Train Data Rows:    19275
Train Data Columns: 9
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (17.344, 0.0, 5.47703, 3.21758)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators

In [ ]:
y_pred = predictor.predict(test_data)

In [ ]:
submit = pd.read_csv('/content/sample_submission.csv')

In [ ]:
submit['target'] = y_pred

In [ ]:
submit.max()

id        TEST_19274
target     14.286489
dtype: object

In [ ]:
submit.to_csv('./auto_10.csv', index=False)

느낀점 
* 기술적인 것에만 집중하다 보니 내가 분석을 하는 목적을 잃어버린 기분이었다. 

